In [ ]:
import matplotlib,numpy,astropy
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from astroquery.vizier import Vizier
from astropy.table import QTable, join
from astropy.table import Table
from astropy import units
from astropy.coordinates import SkyCoord

In [ ]:
font = {'size'   : 16, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1
#plt.rcParams['text.usetex'] = True

In [ ]:
unresolved_list = [38,49,57,62,89,95]
falsegroup_list = [48]
baddata_list = [89]

In [ ]:
HCGs_VLA_HI = Table.read('../output/HCGs_VLA_HI.vo', format='votable')
HCGs_VLA_HI.add_index('HCG')

HCGs_global_HI = Table.read('../output/HCGs_global_HI.vo', format='votable')
HCGs_global_HI.add_index('HCG')

HCGs_HI = astropy.table.join(HCGs_VLA_HI,HCGs_global_HI,keys=['HCG'])
HCGs_HI.add_index('HCG')

HCG_mems = Table.read('../output/HCG_members.vo',format='votable')
HCG_mems.add_index('HCG')

In [ ]:
for HCG in falsegroup_list:
    inx = list(HCGs_VLA_HI['HCG']).index(HCG)
    HCGs_VLA_HI.remove_row(inx)
    inx = list(HCGs_global_HI['HCG']).index(HCG)
    HCGs_global_HI.remove_row(inx)
    inx = list(HCGs_HI['HCG']).index(HCG)
    HCGs_HI.remove_row(inx)
    
for HCG in baddata_list:
    inx = list(HCGs_VLA_HI['HCG']).index(HCG)
    HCGs_VLA_HI.remove_row(inx)
    inx = list(HCGs_global_HI['HCG']).index(HCG)
    HCGs_global_HI.remove_row(inx)
    inx = list(HCGs_HI['HCG']).index(HCG)
    HCGs_HI.remove_row(inx)

In [ ]:
HCGs_HI['Nmem'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='int')

for HCG in HCGs_HI['HCG']:
    HCGs_HI.loc[HCG]['Nmem'] = len(HCG_mems.loc[HCG])

In [ ]:
HCGs_HI['logMHI_p'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')
HCGs_HI['HIphase'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')

HCG_mems_HI = Table.read('../output/HCG_mems_VLA_HI.vo',format='votable')

for HCG in list(HCGs_HI['HCG']):
    
    HCGs_HI.loc[HCG]['logMHI_p'] = r'$'+str(HCGs_HI.loc[HCG]['logMHI_pred_1'])+r' \pm '+str(HCGs_HI.loc[HCG]['e_logMHI_pred_1'])+r'$'
    
    count = numpy.sum(numpy.array(numpy.isfinite(HCG_mems_HI[HCG_mems_HI['HCG'] == HCG]['logMHI']),dtype='int'))
    
    if HCGs_HI.loc[HCG]['ex_frac'] > 0.75 or (hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_HI.loc[HCG]['ex_frac'],'mask')):
        HCGs_HI.loc[HCG]['HIphase'] = '3a'
    elif HCGs_HI.loc[HCG]['ex_frac'] > 0.25:
        HCGs_HI.loc[HCG]['HIphase'] = '2'
    elif count > 1:
        HCGs_HI.loc[HCG]['HIphase'] = '1'
    else:
        HCGs_HI.loc[HCG]['HIphase'] = '3b'
    
    
    if HCG in falsegroup_list:
        HCGs_HI.loc[HCG]['HIphase'] = ''
    elif HCG in unresolved_list and not hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask'):
        HCGs_HI.loc[HCG]['HIphase'] = ''
    elif hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_HI.loc[HCG]['logMHI_GBT'],'mask'):
        HCGs_HI.loc[HCG]['HIphase'] = '3a_nondet'

In [ ]:
plt.figure(figsize=(10,8))

N_nondet = len(HCGs_HI[HCGs_HI['HIphase'] == '3a_nondet'])

plt.scatter(HCGs_HI[HCGs_HI['HIphase'] == '1']['HIdef'],HCGs_HI[HCGs_HI['HIphase'] == '1']['ex_frac'],c='b',label='Phase 1')
plt.scatter(HCGs_HI[HCGs_HI['HIphase'] == '2']['HIdef'],HCGs_HI[HCGs_HI['HIphase'] == '2']['ex_frac'],c='g',label='Phase 2')
plt.scatter(HCGs_HI[HCGs_HI['HIphase'] == '3a']['HIdef'],HCGs_HI[HCGs_HI['HIphase'] == '3a']['ex_frac'],c='r',label='Phase 3a')
plt.scatter(1.5+numpy.zeros(N_nondet),numpy.linspace(0.92,1.08,N_nondet),c='r')
plt.errorbar(1.5+numpy.zeros(N_nondet),numpy.linspace(0.92,1.08,N_nondet),c='r',xlolims=numpy.ones(N_nondet),xerr=0.05,lw=0,elinewidth=1)
plt.scatter(HCGs_HI[HCGs_HI['HIphase'] == '3b']['HIdef'],HCGs_HI[HCGs_HI['HIphase'] == '3b']['ex_frac'],c='magenta',label='Phase 3b')

plt.axhline(0.25,lw=1,c='k',ls='dashed')
plt.axhline(0.75,lw=1,c='k',ls='dashed')
plt.ylim(-0.05,1.099)
plt.xlim(-0.5,1.6)

style = "Simple, tail_width=10, head_width=25, head_length=20"

a1 = patches.FancyArrowPatch(posA=(0., 0.), posB=(0.,0.25),color='b', lw=0, arrowstyle=style,alpha=0.25)
a2 = patches.FancyArrowPatch(posA=(0.,0.25), posB=(0.,0.75),color='g', lw=0, arrowstyle=style,alpha=0.25)
a3 = patches.FancyArrowPatch(posA=(0.,0.75), posB=(0.55,0.975),color='r',connectionstyle="arc3,rad=-.3", lw=0, arrowstyle=style,alpha=0.25)
a4 = patches.FancyArrowPatch(posA=(0.6,0.975), posB=(0.6,0.0125),color='magenta',connectionstyle="arc3,rad=-1.2", lw=0, arrowstyle=style,alpha=0.25)

plt.gca().add_patch(a1)
plt.gca().add_patch(a2)
plt.gca().add_patch(a3)
plt.gca().add_patch(a4)

plt.xlabel('HI-deficiency [dex]')
plt.ylabel(r'Extended $M_\mathrm{HI}$ / Total $M_\mathrm{HI}$')

plt.legend(loc=7)

#plt.savefig('HCG_tidal_frac.png')
#plt.savefig('HCG_tidal_frac.pdf')

In [ ]:
HCGs_WISE = Vizier(catalog='J/ApJ/821/113/table1', columns=['Group', 'Name', 'Class', '_RA', '_DE'], row_limit=-1).query_constraints()[0]
#HCG_pos.add_index('')

In [ ]:
HCGs_HI['IR_class'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')

for HCG in list(HCGs_HI['HCG']):
    namestr = 'HCG'+str(HCG).zfill(2)
    
    tmp = HCGs_WISE[HCGs_WISE['Group'] == namestr]
    
    classes = list(tmp['Class'])
    
    N_active = classes.count('Active')
    N_quiet = classes.count('Quiescent')
    N_canyon = classes.count('Canyon')
    
    if float(N_active)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5:
        HCGs_HI.loc[HCG]['IR_class'] = 'Active'
    elif float(N_quiet+N_canyon)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5:
        HCGs_HI.loc[HCG]['IR_class'] = 'Quiescent'
    else:
        HCGs_HI.loc[HCG]['IR_class'] = 'Intermediate'

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_HI[HCGs_HI['IR_class'] == 'Active']['HIdef'],HCGs_HI[HCGs_HI['IR_class'] == 'Active']['ex_frac'],c='b',label='Active')
plt.scatter(HCGs_HI[HCGs_HI['IR_class'] == 'Intermediate']['HIdef'],HCGs_HI[HCGs_HI['IR_class'] == 'Intermediate']['ex_frac'],c='grey',label='Intermediate')
plt.scatter(HCGs_HI[HCGs_HI['IR_class'] == 'Quiescent']['HIdef'],HCGs_HI[HCGs_HI['IR_class'] == 'Quiescent']['ex_frac'],c='orange',label='Quiescent')

plt.axhline(0.25,lw=1,c='k',ls='dashed')
plt.axhline(0.75,lw=1,c='k',ls='dashed')
plt.ylim(0.,1.)
plt.xlim(-0.5,1.6)

plt.xlabel('HI-deficiency [dex]')
plt.ylabel(r'Extended $M_\mathrm{HI}$ / Total $M_\mathrm{HI}$')

plt.legend(loc=7)

In [ ]:
plt.figure(figsize=(10,8))

N_nondet = len(HCGs_HI[HCGs_HI['HIphase'] == '3a_nondet'])
counter_nondet = 0
exfrac_nondet = numpy.linspace(1.,1.08,N_nondet)

for HCG in list(HCGs_HI['HCG']):
    
    if HCGs_HI.loc[HCG]['IR_class'] == 'Active':
        symbol = '*'
        size = 75
    elif HCGs_HI.loc[HCG]['IR_class'] == 'Quiescent':
        symbol = 'o'
        size = 50
    else:
        symbol = '^'
        size = 50
        
    if HCGs_HI.loc[HCG]['HIphase'] == '1':
        col = 'b'
    elif HCGs_HI.loc[HCG]['HIphase'] == '2':
        col = 'g'
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a':
        col = 'r'
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a_nondet':
        
        if not hasattr(HCGs_HI.loc[HCG]['logMHI_GBT'],'mask'):
            HIdef_val = HCGs_HI.loc[HCG]['HIdef_GBT']
            plt.scatter(HIdef_val,1.,c='r',marker=symbol,s=size)
            if len(HCG_mems.loc[HCG]) == 3:
                plt.scatter(HIdef_val,1.,edgecolor='purple',marker='o',s=200, ls='--',facecolor='none')
        else:
            HIdef_val = 1.5
            plt.scatter(HIdef_val,exfrac_nondet[counter_nondet],c='r',marker=symbol,s=size)
            plt.errorbar(HIdef_val,exfrac_nondet[counter_nondet],c='r',xlolims=True,xerr=0.05,lw=0,elinewidth=1)
            if len(HCG_mems.loc[HCG]) == 3:
                plt.scatter(HIdef_val,exfrac_nondet[counter_nondet],edgecolor='purple',marker='o',s=200, ls='--',facecolor='none')
        counter_nondet += 1
        continue
    else:
        col = 'magenta'
    
    plt.scatter(HCGs_HI.loc[HCG]['HIdef'],HCGs_HI.loc[HCG]['ex_frac'],c=col,marker=symbol,s=size)
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_HI.loc[HCG]['HIdef'],HCGs_HI.loc[HCG]['ex_frac'],edgecolor='purple',marker='o',s=200, ls='--',facecolor='none')
    
style = "Simple, tail_width=10, head_width=25, head_length=20"

a1 = patches.FancyArrowPatch(posA=(0., 0.), posB=(0.,0.25),color='b', lw=0, arrowstyle=style,alpha=0.25)
a2 = patches.FancyArrowPatch(posA=(0.,0.25), posB=(0.,0.75),color='g', lw=0, arrowstyle=style,alpha=0.25)
a3 = patches.FancyArrowPatch(posA=(0.,0.775), posB=(1.0,1.0),color='r',connectionstyle="arc3,rad=-.2", lw=0, arrowstyle=style,alpha=0.25)
a4 = patches.FancyArrowPatch(posA=(1.5,0.9), posB=(0.8,0.0125),color='magenta',connectionstyle="arc3,rad=-0.3", lw=0, arrowstyle=style,alpha=0.25)

plt.gca().add_patch(a1)
plt.gca().add_patch(a2)
plt.gca().add_patch(a3)
plt.gca().add_patch(a4)
    
plt.axhline(0.25,lw=1,c='k',ls='dashed')
plt.axhline(0.75,lw=1,c='k',ls='dashed')
plt.ylim(-0.05,1.05)
plt.xlim(-0.5,1.6)

plt.scatter([],[],c='b',marker='s',s=50,label='Phase 1')
plt.scatter([],[],c='g',marker='s',s=50,label='Phase 2')
plt.scatter([],[],c='r',marker='s',s=50,label='Phase 3a')
plt.scatter([],[],c='magenta',marker='s',s=50,label='Phase 3b')
plt.scatter([],[],c='grey',marker='*',s=75,label='Active')
plt.scatter([],[],c='grey',marker='^',s=50,label='Intermediate')
plt.scatter([],[],c='grey',marker='o',s=50,label='Quiescent')
plt.scatter([],[],edgecolor='purple',marker='o',s=200, ls='--',facecolor='none',label='Triplet')

plt.legend(loc=7)

plt.xlabel('HI-deficiency [dex]')
plt.ylabel(r'Extended $M_\mathrm{HI}$ / Total $M_\mathrm{HI}$')

plt.savefig('HCG_tidal_frac.png',bbox_inches='tight')
plt.savefig('HCG_tidal_frac.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_VLA_HI['logMHI_pred'],HCGs_VLA_HI['logMHI_tot'],c='k',marker='+',s=80,lw=1.5)
m_tmp = numpy.arange(7.35,11.01,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
plt.plot(m_tmp,m_tmp+0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp-0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp+2.*0.13,lw=1,c='k',ls='dotted')
plt.plot(m_tmp,m_tmp-2.*0.13,lw=1,c='k',ls='dotted')

for HCG in HCGs_VLA_HI['HCG']:
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        v_wid_fid = 100. #km/s
        detlim = numpy.log10(3.*(0.001*HCGs_global_HI.loc[HCG]['VLA_rms']/numpy.sqrt(v_wid_fid/abs(HCGs_global_HI.loc[HCG]['chan_wid'])))*v_wid_fid*2.356E5*HCGs_VLA_HI.loc[HCG]['dist']**2.)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',marker='+',s=80.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and not hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='k',marker='+',s=80,lw=1.5)
        
for HCG in HCGs_VLA_HI['HCG']:
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_VLA_HI.loc[HCG]['logMHI_pred'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']-HCGs_VLA_HI.loc[HCG]['HIdef'],
                    edgecolor='purple',marker='o',s=150, ls='--',facecolor='none')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (predicted)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (measured)')
plt.xlim(8.75,11.)
plt.ylim(7.35,11.)

plt.savefig('HCG_def.png',bbox_inches='tight')
plt.savefig('HCG_def.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_VLA_HI['logMHI_pred_HG84'],HCGs_VLA_HI['logMHI_tot'],c='g',marker='o',s=50,lw=1.5,label='HG84 relation')
plt.scatter(HCGs_VLA_HI['logMHI_pred'],HCGs_VLA_HI['logMHI_tot'],c='k',marker='+',s=80,lw=1.5,label='Jones+18 relation')
m_tmp = numpy.arange(7.35,11.01,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
plt.plot(m_tmp,m_tmp+0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp-0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp+2.*0.13,lw=1,c='k',ls='dotted')
plt.plot(m_tmp,m_tmp-2.*0.13,lw=1,c='k',ls='dotted')

for HCG in HCGs_VLA_HI['HCG']:
    plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],
             [HCGs_VLA_HI.loc[HCG]['logMHI_tot'],HCGs_VLA_HI.loc[HCG]['logMHI_tot']],lw=1,c='g',ls='--')
    
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        v_wid_fid = 100. #km/s
        detlim = numpy.log10(3.*(0.001*HCGs_global_HI.loc[HCG]['VLA_rms']/numpy.sqrt(v_wid_fid/abs(HCGs_global_HI.loc[HCG]['chan_wid'])))*v_wid_fid*2.356E5*HCGs_VLA_HI.loc[HCG]['dist']**2.)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',marker='+',s=80.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],detlim,c='g',marker='o',s=50.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],detlim,c='g',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],[detlim,detlim],lw=1,c='g',ls='--')
        
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and not hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='k',marker='+',s=80,lw=1.5)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='g',marker='o',s=50,lw=1.5)
        plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],
                 [HCGs_global_HI.loc[HCG]['logMHI_GBT'],HCGs_global_HI.loc[HCG]['logMHI_GBT']],lw=1,c='g',ls='--')
        
for HCG in HCGs_VLA_HI['HCG']:
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_VLA_HI.loc[HCG]['logMHI_pred'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']-HCGs_VLA_HI.loc[HCG]['HIdef'],
                    edgecolor='purple',marker='o',s=150, ls='--',facecolor='none')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (predicted)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (measured)')
plt.xlim(8.75,11.)
plt.ylim(7.35,11.)

plt.legend()

plt.savefig('HCG_def_HG84comp.png',bbox_inches='tight')
plt.savefig('HCG_def_HG84comp.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_global_HI['logMHI_GBT'],HCGs_global_HI['logMHI_VLA_inGBTbeam'],c='k')
m_tmp = numpy.arange(5.5,10.76,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
#plt.plot(m_tmp,m_tmp-0.25,lw=1,c='k',ls='--')
plt.xlabel('log M_HI (GBT)')
plt.ylabel('log M_HI (VLA weighted to GBT beam)')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (GBT)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (VLA within GBT beam)')
plt.xlim(8,10.75)
plt.ylim(5.75,10.75)

plt.savefig('HCG_GBT_VLA_HImass.png',bbox_inches='tight')
plt.savefig('HCG_GBT_VLA_HImass.pdf',bbox_inches='tight')